# SciGRID Exploration

This notebook explores the SciGRID power system model using pandas.
It is part of the Smart Wires Products and Solutions Analytics
[open training course](https://github.com/smartwiresinc/training).

## 1. Import pandas

Load the Python Data Analysis library with the name `pd`

In [217]:
import pandas as pd

## 2. Load and display data sets

### Vertices, or terminal nodes (substations, generators)

In [218]:
terminals = pd.read_csv('vertices_de_power_160718.csv', quotechar="'")
terminals.head()

,v_id,lon,lat,typ,voltage,frequency,name,operator,ref,wkt_srid_4326
0,1,9.522576,52.360409,substation,220000;110000,50,Hannover/West,TenneT;EON_Netz,NaN,SRID=4326;POINT(9.52257601112577 52.3604090734...
1,2,9.113210,52.543853,substation,220000,50,Umspannwerk Kraftwerk Robert Frank,TenneT,NaN,SRID=4326;POINT(9.11321011483187 52.5438533448...
2,3,9.389745,52.026313,substation,380000;110000,50,UW Grohnde 380 kV / 110 kV,TenneT,NaN,SRID=4326;POINT(9.38974515063535 52.0263130791...
3,4,9.125266,52.538264,substation,380000;220000,50,Landesbergen,TenneT TSO GmbH,NaN,SRID=4326;POINT(9.12526574244048 52.5382641233...
4,5,10.366275,52.284647,substation,380000;220000,50,Wahle,TenneT,NaN,SRID=4326;POINT(10.366274939217 52.2846467643429)


### Links, or branches connecting terminal nodes (transmission lines)

In [219]:
branches = pd.read_csv('links_de_power_160718.csv', quotechar="'")
branches.head()

,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326
0,1,1,2,220000,3.0,2.0,50.0,Leitung 2016,TenneT,2016,43396,0.0800,0.320,11.5,1.3,67129,SRID=4326;LINESTRING(9.52257601112577 52.36040...
1,2,3,4,380000,3.0,4.0,50.0,Leitung 3011,TenneT,NaN,72714,0.0250,0.250,13.7,2.6,67131,SRID=4326;LINESTRING(9.38974515063535 52.02631...
2,3,5,6,220000,3.0,2.0,50.0,Leitung 2024,TenneT,1,33954,0.0800,0.320,11.5,1.3,67151,SRID=4326;LINESTRING(10.366274939217 52.284646...
3,4,7,5,380000,3.0,4.0,50.0,Leitung 3026,TenneT,3026,33483,0.0250,0.250,13.7,2.6,67153,SRID=4326;LINESTRING(9.9171797569037 52.278168...
4,5,8,9,380000,6.0,4.0,50.0,KKW Krümmel - UW Lüneburg,TenneT,NaN,28172,0.0125,0.125,27.4,5.2,67585,SRID=4326;LINESTRING(10.4149924241538 53.41260...


## 3. Try Jupyter

Create a dataframe with substations only.

In [220]:
substations = terminals[terminals["typ"] == "substation"]
substations.head()

,v_id,lon,lat,typ,voltage,frequency,name,operator,ref,wkt_srid_4326
0,1,9.522576,52.360409,substation,220000;110000,50,Hannover/West,TenneT;EON_Netz,NaN,SRID=4326;POINT(9.52257601112577 52.3604090734...
1,2,9.113210,52.543853,substation,220000,50,Umspannwerk Kraftwerk Robert Frank,TenneT,NaN,SRID=4326;POINT(9.11321011483187 52.5438533448...
2,3,9.389745,52.026313,substation,380000;110000,50,UW Grohnde 380 kV / 110 kV,TenneT,NaN,SRID=4326;POINT(9.38974515063535 52.0263130791...
3,4,9.125266,52.538264,substation,380000;220000,50,Landesbergen,TenneT TSO GmbH,NaN,SRID=4326;POINT(9.12526574244048 52.5382641233...
4,5,10.366275,52.284647,substation,380000;220000,50,Wahle,TenneT,NaN,SRID=4326;POINT(10.366274939217 52.2846467643429)


The number of substations is

In [221]:
substations.shape[0]

437

Number of substations per voltage level

In [222]:
A = substations.groupby(by='voltage').size() # A is a Series element with number of substations per voltage level
A = A[A.index.str.contains(';') == False]
A = A[A.index.str.contains(',') == False]
A

voltage
110000     3
150000     2
220000    40
380000    43
400000     4
dtype: int64

The most common voltage level is

In [223]:
A.loc[A == A.max()]

voltage
380000    43
dtype: int64

The number of unique operators is

In [224]:
len(pd.merge(branches,terminals,how='outer',on='operator').groupby(by='operator'))

121

Using an alternative method, the number of unique operators can be calculated as

In [225]:
operators = pd.concat([branches.operator,terminals.operator])
operators = operators.groupby(operators)
len(operators)

121

The number of transmission lines longer than 10km is 

In [226]:
branches[branches.length_m > 10000].shape[0]

633

In [227]:
# C is a Series element with the number of lines per resistance/km value
C = branches.groupby(by='r_ohmkm').size().sort_values(ascending=False) 

The two most common resistance/km values are

In [228]:
C.head(2)

r_ohmkm
0.025    226
0.080    211
dtype: int64

Adding a column with all line's total resistance

In [229]:
branches['total_r'] = branches['r_ohmkm'] * branches['length_m'] / 1000

The line with the maximum total resistance is

In [230]:
branches.sort_values(ascending = False, by = 'total_r').head(1)

,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326,total_r
495,496,282,342,220000,3.0,1.0,NaN,NaN,50Hertz,358,158679,0.16,0.64,5.75,0.65,3739954,SRID=4326;LINESTRING(12.518222136544 51.157072...,25.38864


Adding a column with X/R ratios

In [231]:
branches['XtoR'] = branches['x_ohmkm'] / branches['r_ohmkm']

The lines with the 10 largest X/R ratio are given by

In [232]:
import time
start = time.time()
D1=branches.sort_values(ascending=False, by='XtoR').head(10)
print("%s seconds" % (time.time() - start))
D1

0.002572298049926758 seconds


,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326,total_r,XtoR
518,519,15,337,380000,9.0,4.0,NaN,NaN,NaN,443;444;446,61219,0.008333,0.083333,41.100,7.80,3749855,SRID=4326;LINESTRING(11.868156362746 48.206802...,0.510138,10.00036
835,836,520,521,380000,3.0,3.0,50.0,NaN,50Hertz,578,17414,0.033333,0.333333,10.275,1.95,5308496,SRID=4326;LINESTRING(12.881244953781 50.518645...,0.580461,10.00009
465,466,279,65,380000,3.0,3.0,50.0,NaN,NaN,445,81644,0.033333,0.333333,10.275,1.95,3730415,SRID=4326;LINESTRING(12.8139099558581 50.85791...,2.721439,10.00009
443,444,307,311,380000,3.0,3.0,50.0,Kupferzell - Großgartach,NaN,NaN,47743,0.033333,0.333333,10.275,1.95,3694016,SRID=4326;LINESTRING(9.15135403252636 49.14338...,1.591417,10.00009
73,74,10,74,380000,3.0,3.0,50.0,521,50Hertz,521,130600,0.033333,0.333333,10.275,1.95,918563,SRID=4326;LINESTRING(12.9416466491702 52.56211...,4.353290,10.00009
71,72,18,40,380000,3.0,3.0,50.0,509,50Hertz,509,56402,0.033333,0.333333,10.275,1.95,912407,SRID=4326;LINESTRING(13.6834769799404 54.13909...,1.880048,10.00009
70,71,73,72,380000,3.0,3.0,50.0,503,50Hertz,503,110042,0.033333,0.333333,10.275,1.95,907513,SRID=4326;LINESTRING(13.1570905224719 51.82845...,3.668030,10.00009
67,68,37,72,380000,3.0,3.0,50.0,535 Lauchstädt - Wolmirstedt,50Hertz,535,156923,0.033333,0.333333,10.275,1.95,904160,SRID=4326;LINESTRING(11.6389126099956 52.26873...,5.230714,10.00009
65,66,37,75,380000,3.0,3.0,50.0,536 Klostermansfeld - Wolmirstedt,50Hertz,536,118963,0.033333,0.333333,10.275,1.95,904158,SRID=4326;LINESTRING(11.6389126099956 52.26873...,3.965394,10.00009
64,65,73,74,380000,3.0,3.0,50.0,501,50Hertz,501,54235,0.033333,0.333333,10.275,1.95,902597,SRID=4326;LINESTRING(13.1570905224719 51.82845...,1.807815,10.00009


In [233]:
start = time.time()
D2=branches[['l_id','XtoR']].sort_values(ascending=False, by='XtoR').head(10)
print("%s seconds" % (time.time() - start))
D2

0.0036988258361816406 seconds


,l_id,XtoR
518,519,10.00036
835,836,10.00009
465,466,10.00009
443,444,10.00009
73,74,10.00009
71,72,10.00009
70,71,10.00009
67,68,10.00009
65,66,10.00009
64,65,10.00009


The lines with the 10 smallest X/R ratio are given by

In [234]:
start = time.time()
D3=branches.sort_values(ascending=True, by='XtoR').head(10)
print("%s seconds" % (time.time() - start))
D3

0.0031681060791015625 seconds


,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326,total_r,XtoR
0,1,1,2,220000,3.0,2.0,50.0,Leitung 2016,TenneT,2016,43396,0.08,0.32,11.50,1.30,67129,SRID=4326;LINESTRING(9.52257601112577 52.36040...,3.47168,4.0
663,664,24,272,220000,3.0,1.0,50.0,NaN,TenneT,206,63332,0.16,0.64,5.75,0.65,5207200,SRID=4326;LINESTRING(9.72699275106814 54.29142...,10.13312,4.0
664,665,13,14,220000,3.0,2.0,50.0,Oba-Nfi II,TenneT TSO GmbH,264,39295,0.08,0.32,11.50,1.30,5247406,SRID=4326;LINESTRING(11.3704263094168 48.29149...,3.14360,4.0
270,271,69,215,220000,3.0,2.0,50.0,Anilin 2B,Amprion,NaN,13609,0.08,0.32,11.50,1.30,1636726,SRID=4326;LINESTRING(8.41937108566179 49.63373...,1.08872,4.0
267,268,69,215,220000,3.0,2.0,50.0,Anilin 1A,Amprion,NaN,13611,0.08,0.32,11.50,1.30,1636721,SRID=4326;LINESTRING(8.41937108566179 49.63373...,1.08888,4.0
266,267,213,214,220000,3.0,1.0,50.0,Rheinau Ost,Amprion,NaN,45094,0.16,0.64,5.75,0.65,1636720,SRID=4326;LINESTRING(8.58869053015616 49.81551...,7.21504,4.0
681,682,362,404,220000,3.0,2.0,50.0,NaN,NaN,NaN,15077,0.08,0.32,11.50,1.30,5716121,SRID=4326;LINESTRING(8.04936553586669 47.55087...,1.20616,4.0
683,684,508,509,220000,3.0,2.0,50.0,NaN,NaN,NaN,29715,0.08,0.32,11.50,1.30,5978211,SRID=4326;LINESTRING(7.97117863459035 47.54578...,2.37720,4.0
684,685,362,508,220000,3.0,2.0,50.0,NaN,NaN,NaN,6386,0.08,0.32,11.50,1.30,5978214,SRID=4326;LINESTRING(8.04936553586669 47.55087...,0.51088,4.0
273,274,213,214,220000,3.0,2.0,50.0,Rheinau West,Amprion,NaN,45094,0.08,0.32,11.50,1.30,1636737,SRID=4326;LINESTRING(8.58869053015616 49.81551...,3.60752,4.0


In [235]:
start = time.time()
D4=branches[['l_id','XtoR']].sort_values(ascending=True, by='XtoR').head(10)
print("%s seconds" % (time.time() - start))
D4

0.003247499465942383 seconds


,l_id,XtoR
0,1,4.0
663,664,4.0
664,665,4.0
270,271,4.0
267,268,4.0
266,267,4.0
681,682,4.0
683,684,4.0
684,685,4.0
273,274,4.0


Adding a column with lines' total impedance (z_ohm)

In [236]:
branches['z_ohm'] = ((branches['x_ohmkm'].pow(2) + branches['r_ohmkm'].pow(2)).pow(1./2)) * branches['length_m'] / 1000

The maximum line impedance is

In [237]:
max_Z=branches['z_ohm'].max()
max_Z

104.68004441078158

The line with the maximum total impedance is

In [238]:
branches[branches['z_ohm'] == max_Z]

,l_id,v_id_1,v_id_2,voltage,cables,wires,frequency,name,operator,ref,length_m,r_ohmkm,x_ohmkm,c_nfkm,i_th_max_a,from_relation,wkt_srid_4326,total_r,XtoR,z_ohm
495,496,282,342,220000,3.0,1.0,NaN,NaN,50Hertz,358,158679,0.16,0.64,5.75,0.65,3739954,SRID=4326;LINESTRING(12.518222136544 51.157072...,25.38864,4.0,104.680044
